In [248]:
import yaml
import os
import sys

In [249]:
import pandas as pd

In [250]:
# add module path to the python path
# module_path = os.path.abspath(os.path.join('../src'))
# sys.path.insert(0, module_path)

# add project root to python path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.join(notebook_dir, '..')
sys.path.insert(0, project_root)

from src import util

In [251]:
with open('../config.yaml') as file:
    config = yaml.safe_load(file)
config

{'raw_data': {'sample': '../data/raw/Sample-Superstore.csv'},
 'processed_data': {'database': '../data/processed/vs_group_order-db.db'}}

# Import

In [252]:
# detect file encoding
# temporarily needed: does not have to run each time
# encoding, confidence = util.detect_encoding(config['raw_data']['sample'])

# print('encoding: ', encoding)
# print('confidence: ', confidence)

In [253]:
df = pd.read_csv(config['raw_data']['sample'], encoding='Windows-1252')

In [254]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# Normalize

In [255]:
df.columns = df.columns.str.replace(' ', '').str.replace('-', '')

In [256]:
df = df.drop(columns='RowID')

In [257]:
#df = df.drop_duplicates()

In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OrderID       9994 non-null   object 
 1   OrderDate     9994 non-null   object 
 2   ShipDate      9994 non-null   object 
 3   ShipMode      9994 non-null   object 
 4   CustomerID    9994 non-null   object 
 5   CustomerName  9994 non-null   object 
 6   Segment       9994 non-null   object 
 7   Country       9994 non-null   object 
 8   City          9994 non-null   object 
 9   State         9994 non-null   object 
 10  PostalCode    9994 non-null   int64  
 11  Region        9994 non-null   object 
 12  ProductID     9994 non-null   object 
 13  Category      9994 non-null   object 
 14  SubCategory   9994 non-null   object 
 15  ProductName   9994 non-null   object 
 16  Sales         9994 non-null   float64
 17  Quantity      9994 non-null   int64  
 18  Discount      9994 non-null 

# Set Data Types

In [259]:
df['OrderID']       = df['OrderID'].astype(pd.StringDtype())
df['OrderDate']     = pd.to_datetime(df['OrderDate']).dt.floor('D')
df['ShipDate']      = pd.to_datetime(df['ShipDate']).dt.floor('D')
df['ShipMode']      = df['ShipMode'].astype('category')
df['CustomerID']    = df['CustomerID'].astype(pd.StringDtype())
df['CustomerName']  = df['CustomerName'].astype(pd.StringDtype())
df['Segment']       = df['Segment'].astype('category')
df['Country']       = df['Country'].astype('category')
df['City']          = df['City'].astype('category')
df['State']         = df['State'].astype('category')
df['Region']        = df['Region'].astype('category')
df['ProductID']     = df['ProductID'].astype(pd.StringDtype())
df['Category']      = df['Category'].astype('category')
df['SubCategory']   = df['SubCategory'].astype('category')
df['ProductName']   = df['ProductName'].astype(pd.StringDtype())

In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   OrderID       9994 non-null   string        
 1   OrderDate     9994 non-null   datetime64[ns]
 2   ShipDate      9994 non-null   datetime64[ns]
 3   ShipMode      9994 non-null   category      
 4   CustomerID    9994 non-null   string        
 5   CustomerName  9994 non-null   string        
 6   Segment       9994 non-null   category      
 7   Country       9994 non-null   category      
 8   City          9994 non-null   category      
 9   State         9994 non-null   category      
 10  PostalCode    9994 non-null   int64         
 11  Region        9994 non-null   category      
 12  ProductID     9994 non-null   string        
 13  Category      9994 non-null   category      
 14  SubCategory   9994 non-null   category      
 15  ProductName   9994 non-null   string  

# Making sense of the data

## Meaning of Order ID

In [261]:
df.OrderID.value_counts()

OrderID
CA-2017-100111    14
CA-2017-157987    12
CA-2016-165330    11
US-2016-108504    11
CA-2015-131338    10
                  ..
US-2015-131842     1
CA-2016-144939     1
CA-2015-157035     1
CA-2016-168361     1
CA-2017-119914     1
Name: count, Length: 5009, dtype: Int64

In [262]:
df[df.OrderID == 'CA-2017-100111']

,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
6090,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,FUR-CH-10003846,Furniture,Chairs,Hon Valutask Swivel Chairs,272.646,3,0.1,18.1764
6091,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,TEC-AC-10002647,Technology,Accessories,Logitech Wireless Boombox Speaker - portable -...,212.800,2,0.0,95.7600
6092,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-PA-10000807,Office Supplies,Paper,"TOPS ""Important Message"" Pads, Canary, 4-1/4 x...",38.520,9,0.0,18.1044
6093,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,TEC-AC-10001465,Technology,Accessories,SanDisk Cruzer 64 GB USB Flash Drive,72.640,2,0.0,21.7920
6094,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-ST-10000615,Office Supplies,Storage,"SimpliFile Personal File, Black Granite, 15w x...",45.400,4,0.0,12.7120
6095,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-PA-10002713,Office Supplies,Paper,"Adams Phone Message Book, 200 Message Capacity...",13.760,2,0.0,6.3296
6096,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,FUR-CH-10003061,Furniture,Chairs,"Global Leather Task Chair, Black",80.991,1,0.1,8.0991
6097,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",11.784,3,0.2,3.9771
6098,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-FA-10000304,Office Supplies,Fasteners,Advantus Push Pins,4.360,2,0.0,1.7876
6099,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,FUR-CH-10004086,Furniture,Chairs,Hon 4070 Series Pagoda Armless Upholstered Sta...,2888.127,11,0.1,609.7157


In [263]:
df[df.OrderID == 'CA-2015-131338']

,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
1579,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,TEC-PH-10003012,Technology,Phones,Nortel Meridian M3904 Professional Digital phone,307.980,2,0.0,89.3142
1580,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,382.806,9,0.4,-153.1224
1581,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-ST-10000642,Office Supplies,Storage,"Tennsco Lockers, Gray",41.960,2,0.0,2.9372
1582,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-BI-10000545,Office Supplies,Binders,GBC Ibimaster 500 Manual ProClick Binding System,1217.568,2,0.2,456.5880
1583,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-FU-10002157,Furniture,Furnishings,Artistic Insta-Plaque,47.040,3,0.0,18.3456
1584,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-FU-10001706,Furniture,Furnishings,Longer-Life Soft White Bulbs,6.160,2,0.0,2.9568
1585,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,TEC-PH-10000984,Technology,Phones,Panasonic KX-TG9471B,979.950,5,0.0,274.3860
1586,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-PA-10001357,Office Supplies,Paper,Xerox 1886,143.700,3,0.0,68.9760
1587,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-FA-10000992,Office Supplies,Fasteners,"Acco Clips to Go Binder Clips, 24 Clips in Two...",10.650,3,0.0,5.0055
1588,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,TEC-AC-10002600,Technology,Accessories,Belkin QODE FastFit Bluetooth Keyboard,247.800,4,0.0,34.6920


* OrderID is not unique
* A customer places an order with several products:
  * For each product in an order an additional row is added.
  * Order Date and Shipment Data is typically the same.
  * An Order could however be split into multiple Shipments. 
* OrderID is more an OrderReference.

## Sales and Profit

In [264]:
# What is the meaning of sales? 
df[df.ProductID == 'FUR-TA-10002607']

,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
241,CA-2016-157749,2016-06-04,2016-06-09,Second Class,KL-16645,Ken Lonsdale,Consumer,United States,Chicago,Illinois,60610,Central,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,177.225,5,0.5,-120.5130
746,CA-2014-124429,2014-05-27,2014-05-27,Same Day,MH-17785,Maya Herman,Corporate,United States,San Diego,California,92105,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,567.120,10,0.2,-28.3560
1580,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,382.806,9,0.4,-153.1224
1814,CA-2015-131597,2015-09-14,2015-09-18,Standard Class,SP-20620,Stefania Perrino,Corporate,United States,Los Angeles,California,90045,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,170.136,3,0.2,-8.5068
4294,CA-2017-101581,2017-10-22,2017-10-27,Standard Class,DW-13195,David Wiener,Corporate,United States,Redmond,Oregon,97756,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,177.225,5,0.5,-120.5130
5373,CA-2015-118738,2015-10-24,2015-10-30,Standard Class,AG-10495,Andrew Gjertsen,Corporate,United States,Houston,Texas,77041,Central,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,347.361,7,0.3,-69.4722
6799,CA-2016-109827,2016-12-25,2017-01-01,Standard Class,LW-16825,Laurel Workman,Corporate,United States,Phoenix,Arizona,85023,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,35.445,1,0.5,-24.1026
9031,CA-2017-128041,2017-09-01,2017-09-01,Same Day,RW-19540,Rick Wilson,Corporate,United States,Seattle,Washington,98103,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,283.560,4,0.0,45.3696


In [265]:
283.56/4 * 0.5

35.445

* **Sales**: Total Price of order after discount = (1 - Discount) * Price * Quantity
* **Profit**: Total Profit of order; Profit = (ProfitRate * Price - Discount * Price) * Quantity

# Check Data Integrity

In [266]:
# duplicate rows
print('duplicates', df.duplicated().sum())

duplicates 1


In [267]:
# missing values
df.isna().sum()

OrderID         0
OrderDate       0
ShipDate        0
ShipMode        0
CustomerID      0
CustomerName    0
Segment         0
Country         0
City            0
State           0
PostalCode      0
Region          0
ProductID       0
Category        0
SubCategory     0
ProductName     0
Sales           0
Quantity        0
Discount        0
Profit          0
dtype: int64

no missing values

# Deal with the duplicates in order stem data
order stem data: non-numeric data of an order

In [268]:
# sum up rows where order details are equal, i.e. 'OrderReference', 'CustomerID', 'ProductID', 'OrderDate', 'Discount', 'ShipmentID'
order_stem_data = [
    'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
    'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
    'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName', 'Discount'
    ]

mask_dups = df.duplicated(subset=order_stem_data, keep=False)
print('duplicates', mask_dups.sum())
df[mask_dups]

duplicates 16


,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
350,CA-2016-129714,2016-09-01,2016-09-03,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,New York,10009,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,24.560,2,0.0,11.5432
352,CA-2016-129714,2016-09-01,2016-09-03,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,New York,10009,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,49.120,4,0.0,23.0864
430,US-2016-123750,2016-04-15,2016-04-21,Standard Class,RB-19795,Ross Baird,Home Office,United States,Gastonia,North Carolina,28052,South,TEC-AC-10004659,Technology,Accessories,Imation Secure+ Hardware Encrypted USB 2.0 Fla...,408.744,7,0.2,76.6395
431,US-2016-123750,2016-04-15,2016-04-21,Standard Class,RB-19795,Ross Baird,Home Office,United States,Gastonia,North Carolina,28052,South,TEC-AC-10004659,Technology,Accessories,Imation Secure+ Hardware Encrypted USB 2.0 Fla...,291.960,5,0.2,54.7425
1300,CA-2016-137043,2016-12-23,2016-12-25,Second Class,LC-17140,Logan Currie,Consumer,United States,Springfield,Virginia,22153,South,FUR-FU-10003664,Furniture,Furnishings,"Electrix Architect's Clamp-On Swing Arm Lamp, ...",572.760,6,0.0,166.1004
1301,CA-2016-137043,2016-12-23,2016-12-25,Second Class,LC-17140,Logan Currie,Consumer,United States,Springfield,Virginia,22153,South,FUR-FU-10003664,Furniture,Furnishings,"Electrix Architect's Clamp-On Swing Arm Lamp, ...",286.380,3,0.0,83.0502
3183,CA-2017-152912,2017-11-09,2017-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,1633.140,9,0.0,473.6106
3184,CA-2017-152912,2017-11-09,2017-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,544.380,3,0.0,157.8702
3405,US-2014-150119,2014-04-23,2014-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
3406,US-2014-150119,2014-04-23,2014-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588


* Some orders are split up into two rows and can be summed up into one row.
* Possible reason: The order has been extended afterwards on the same day.

In [269]:
# sum up
df = df.groupby(by=order_stem_data, observed=True).agg({
    'Sales':'sum', 
    'Quantity':'sum', 
    'Profit':'sum'}
    ).reset_index()

In [270]:
# check again for duplicates
df.duplicated().sum()

0

No duplicates remain after this cleaning step.

# Build database tables
* Build tables for the database
* Apply cleaning steps

In [271]:
customer_col    = ['CustomerID','CustomerName', 'Segment']
product_col     = ['ProductID', 'Category', 'SubCategory', 'ProductName']
address_col     = ['Country', 'Region', 'State', 'City', 'PostalCode']
shipment_col    = ['ShipDate', 'ShipMode']
orders_col      = ['OrderID', 'CustomerID', 'ProductID', 'OrderDate','Sales', 'Quantity', 'Discount', 'Profit']

In [272]:
# merge IDs to dataframe

In [273]:
# map col names to db col names
name_mapping={
    'CustomerName'  : 'Name',
    'ProductName'   : 'Name',
    'OrderID'       : 'OrderReference'
}

## Customer

In [274]:
# create dataframes
customer = df[customer_col].drop_duplicates().set_index('CustomerID').rename(columns=name_mapping)
customer.head()


,Name,Segment
CustomerID,,
DK-13375,Dennis Kane,Consumer
EB-13705,Ed Braxton,Corporate
NF-18475,Neil Französisch,Home Office
JC-15340,Jasper Cacioppo,Consumer
JM-15655,Jim Mitchum,Corporate


In [275]:
print('Customer ID duplicates',     customer.index.duplicated().sum())
print('Customer',                   customer.duplicated().sum())
print('Customer Name duplicates',   customer.Name.duplicated().sum())

Customer ID duplicates 0
Customer 0
Customer Name duplicates 0


## Product

In [276]:
product  = df[product_col].drop_duplicates().set_index('ProductID').rename(columns=name_mapping)
product.head()

,Category,SubCategory,Name
ProductID,,,
TEC-PH-10002075,Technology,Phones,AT&T EL51110 DECT
FUR-TA-10003715,Furniture,Tables,Hon 2111 Invitation Series Corner Table
OFF-BI-10001597,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ..."
OFF-PA-10000176,Office Supplies,Paper,Xerox 1887
OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x..."


### Duplicates

In [277]:
# duplicate product IDs
product[product.index.duplicated(keep=False)].sort_index()

,Category,SubCategory,Name
ProductID,,,
FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases
FUR-BO-10002213,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish"
FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black"
FUR-CH-10001146,Furniture,Chairs,"Global Value Mid-Back Manager's Chair, Gray"
FUR-FU-10001473,Furniture,Furnishings,"Eldon Executive Woodline II Desk Accessories, ..."
...,...,...,...
TEC-PH-10002200,Technology,Phones,Aastra 6757i CT Wireless VoIP phone
TEC-PH-10002310,Technology,Phones,Plantronics Calisto P620-M USB Wireless Speake...
TEC-PH-10002310,Technology,Phones,Panasonic KX T7731-B Digital phone


There are 64 duplicate product IDs.

In [278]:
# What has changed? Only the names of the products?
product_analyse_dups = product.reset_index().copy()
mask_dup_subset = product_analyse_dups.duplicated(subset=['ProductID', 'Category', 'SubCategory'], keep=False)
print('Duplicates without Name: ', mask_dup_subset.sum())

product_analyse_dups[mask_dup_subset].sort_values('ProductID')

Duplicates without Name:  64


,ProductID,Category,SubCategory,Name
550,FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases
1506,FUR-BO-10002213,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish"
1734,FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black"
248,FUR-CH-10001146,Furniture,Chairs,"Global Value Mid-Back Manager's Chair, Gray"
1368,FUR-FU-10001473,Furniture,Furnishings,"Eldon Executive Woodline II Desk Accessories, ..."
...,...,...,...,...
1465,TEC-PH-10002200,Technology,Phones,Aastra 6757i CT Wireless VoIP phone
1533,TEC-PH-10002310,Technology,Phones,Plantronics Calisto P620-M USB Wireless Speake...
564,TEC-PH-10002310,Technology,Phones,Panasonic KX T7731-B Digital phone
245,TEC-PH-10004531,Technology,Phones,AT&T CL2909


* All duplicate product IDs differ only in Name.
* We have a fecking error in the system. Where could these errors come from?
  * The name might have changed.
  * The product ID is not a unique identifier or it might have been set wrong.
* Has the product simply been renamed? Scrolling the data:
  * some names are very similar
  * some are not
  * The products have not been renamed; probably they have been mixed up.
  * Possible further check: Check the dates
* Is the product ID really an ID? Or does it refer to a class of products?
  * From the looks it should refer to a specific product.
  * Logically, for the order to be well defined, all information about the product must be given with the information in the above table.
* Solution
  1. Check the number of orders that are affected.
  2. Check if the order IDs have been mixed up.
  3. Calculate the Price and see if that makes a difference.
  4. Drop the duplicates and only keep 32 products
  5. resolve the errors 

In [279]:
# TODO How many entries are affected in the full data set?

In [280]:
# Are just the product ID mixed up? Are there products with similar names, but different product ID?
product_analyse_dups.loc[product_analyse_dups.duplicated(subset=['Category', 'SubCategory', 'Name'], keep=False)].sort_values(['Name', 'ProductID'])

# TODO How to fix? probably not possible ...
# merge with product_analyse_dups[mask_dup_subset]
# replace 

,ProductID,Category,SubCategory,Name
1127,OFF-EN-10000461,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes"
660,OFF-EN-10000781,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes"
284,OFF-BI-10000829,Office Supplies,Binders,Avery Non-Stick Binders
45,OFF-BI-10004140,Office Supplies,Binders,Avery Non-Stick Binders
1373,OFF-PA-10000249,Office Supplies,Paper,Easy-staple paper
744,OFF-PA-10000349,Office Supplies,Paper,Easy-staple paper
293,OFF-PA-10000474,Office Supplies,Paper,Easy-staple paper
1679,OFF-PA-10000565,Office Supplies,Paper,Easy-staple paper
875,OFF-PA-10001685,Office Supplies,Paper,Easy-staple paper
1684,OFF-PA-10002764,Office Supplies,Paper,Easy-staple paper


* There are 60 duplicate product names, whereas there are products with very generic names, like 'Easy-staple paper'.
* This indicates that the products are not defined until the smallest level of detail and that the product ID refers to a class of products.

* Conclusion: 
  * Keep all Name duplicates.
  * Keep all ProductID duplicates and create a new ProductID (just put the number to a new)
  * Create a new product ID.

### Integrity
* Product ID: cat-subcat-number
* integrity check: check if category and subcategory are ok


In [281]:
# clean up
# product = product[~product.index.duplicated(keep='first')]

### separate product categories

In [282]:
product_all = product.copy()
product_all

,Category,SubCategory,Name
ProductID,,,
TEC-PH-10002075,Technology,Phones,AT&T EL51110 DECT
FUR-TA-10003715,Furniture,Tables,Hon 2111 Invitation Series Corner Table
OFF-BI-10001597,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ..."
OFF-PA-10000176,Office Supplies,Paper,Xerox 1887
OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x..."
...,...,...,...
OFF-AR-10003477,Office Supplies,Art,4009 Highlighters
OFF-LA-10003388,Office Supplies,Labels,Avery 5
OFF-EN-10003448,Office Supplies,Envelopes,"Peel & Seel Recycled Catalog Envelopes, Brown"


In [300]:
product_categories = product_all[['Category', 'SubCategory']].drop_duplicates().reset_index(drop=True)
product = pd.merge(
    product_all.reset_index(),
    product_categories.reset_index(), 
    on=['Category', 'SubCategory'], how='left'
    ).drop(columns=['Category', 'SubCategory']).rename(columns={'index':'ProductCategoryID', 'ProductID':'ProductReference'})
product

,ProductReference,Name,ProductCategoryID
0,TEC-PH-10002075,AT&T EL51110 DECT,0
1,FUR-TA-10003715,Hon 2111 Invitation Series Corner Table,1
2,OFF-BI-10001597,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",2
3,OFF-PA-10000176,Xerox 1887,3
4,OFF-BI-10000343,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",2
...,...,...,...
1889,OFF-AR-10003477,4009 Highlighters,6
1890,OFF-LA-10003388,Avery 5,10
1891,OFF-EN-10003448,"Peel & Seel Recycled Catalog Envelopes, Brown",7
1892,OFF-AP-10002203,Eureka Disposable Bags for Sanitaire Vibra Gro...,12


## Address

In [320]:
address = df[address_col].drop_duplicates().reset_index(drop=True)#.set_index('ProductID').rename(columns=name_mapping)
address.head()

,Country,Region,State,City,PostalCode
0,United States,East,New York,New York City,10024
1,United States,West,California,San Francisco,94122
2,United States,South,Florida,Jacksonville,32216
3,United States,West,Arizona,Glendale,85301
4,United States,East,New York,New York City,10035


### check data integrity

In [321]:
address.Country.value_counts()
address.Region.value_counts()
address.State.value_counts()
# address.City.value_counts()
# address.PostalCode.value_counts()

State
California              90
Texas                   59
Florida                 38
Illinois                35
Ohio                    24
Michigan                23
New York                21
Washington              19
North Carolina          19
New Jersey              18
Colorado                16
Indiana                 15
Tennessee               15
Massachusetts           14
Minnesota               13
Arizona                 13
Virginia                13
Wisconsin               13
Pennsylvania            11
Georgia                 11
Connecticut             10
Utah                    10
Oregon                   9
Arkansas                 9
Missouri                 9
Alabama                  8
Kentucky                 8
Iowa                     8
Maryland                 7
New Mexico               7
Oklahoma                 7
South Carolina           6
Idaho                    6
Montana                  5
Nevada                   5
Louisiana                5
Kansas                

In [322]:
print('nunique ["City", "PostalCode"]: ', address[['City', 'PostalCode']].nunique())
print('duplicates ["City", "PostalCode"]: ', address[['City', 'PostalCode']].duplicated().sum())

nunique ["City", "PostalCode"]:  City          531
PostalCode    631
dtype: int64
duplicates ["City", "PostalCode"]:  0


In [323]:
address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 632 entries, 0 to 631
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Country     632 non-null    category
 1   Region      632 non-null    category
 2   State       632 non-null    category
 3   City        632 non-null    category
 4   PostalCode  632 non-null    int64   
dtypes: category(4), int64(1)
memory usage: 30.2 KB


### build secondary address info table

In [324]:
address_secondary = address[['Country', 'Region', 'State']].drop_duplicates()
address_secondary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 0 to 557
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Country  49 non-null     category
 1   Region   49 non-null     category
 2   State    49 non-null     category
dtypes: category(3)
memory usage: 2.3 KB


In [325]:
# separate secondary address info from address
address_all = address
address = pd.merge(address, address_secondary.reset_index(), on=['Country', 'Region', 'State']).drop(columns=['Country', 'Region', 'State']).rename(columns={'index':'AddressSecondaryID'})

## Shipment

In [301]:
shipment_all = df[shipment_col+address_col].drop_duplicates().reset_index(drop=True)

In [302]:
merge_col = ['Country', 'City', 'State', 'PostalCode','Region']
shipment = pd.merge(
    shipment_all, address_all.reset_index(), how='inner', 
    on=merge_col
    ).rename(columns={'index':'AddressID'}).drop(columns=merge_col)

In [303]:
shipment.head()

,ShipDate,ShipMode,AddressID
0,2014-09-13,Standard Class,0
1,2014-07-12,Standard Class,1
2,2014-03-18,Standard Class,2
3,2014-02-03,Standard Class,0
4,2014-04-15,Standard Class,3


In [304]:
shipment.shape

(4973, 3)

## Orders

In [305]:
orders_all = df[orders_col+shipment_col+address_col]

In [306]:
orders_all.head()

,OrderID,CustomerID,ProductID,OrderDate,Sales,Quantity,Discount,Profit,ShipDate,ShipMode,Country,Region,State,City,PostalCode
0,CA-2014-100006,DK-13375,TEC-PH-10002075,2014-09-07,377.970,3,0.0,109.6113,2014-09-13,Standard Class,United States,East,New York,New York City,10024
1,CA-2014-100090,EB-13705,FUR-TA-10003715,2014-07-08,502.488,3,0.2,-87.9354,2014-07-12,Standard Class,United States,West,California,San Francisco,94122
2,CA-2014-100090,EB-13705,OFF-BI-10001597,2014-07-08,196.704,6,0.2,68.8464,2014-07-12,Standard Class,United States,West,California,San Francisco,94122
3,CA-2014-100293,NF-18475,OFF-PA-10000176,2014-03-14,91.056,6,0.2,31.8696,2014-03-18,Standard Class,United States,South,Florida,Jacksonville,32216
4,CA-2014-100328,JC-15340,OFF-BI-10000343,2014-01-28,3.928,1,0.2,1.3257,2014-02-03,Standard Class,United States,East,New York,New York City,10024


In [307]:
orders_all.duplicated().sum()

0

In [308]:
merge_col = shipment_col+address_col
orders = pd.merge(orders_all, 
         shipment_all.reset_index(), 
         how='left', 
         on=merge_col).drop(columns=merge_col).rename(columns={'index':'ShipmentID'})

In [309]:
orders = orders.rename(columns=name_mapping)

In [310]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9986 entries, 0 to 9985
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OrderReference  9986 non-null   string        
 1   CustomerID      9986 non-null   string        
 2   ProductID       9986 non-null   string        
 3   OrderDate       9986 non-null   datetime64[ns]
 4   Sales           9986 non-null   float64       
 5   Quantity        9986 non-null   int64         
 6   Discount        9986 non-null   float64       
 7   Profit          9986 non-null   float64       
 8   ShipmentID      9986 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), string(3)
memory usage: 702.3 KB


In [311]:
order_prim_key = ['ProductID', 'CustomerID', 'OrderReference', 'ShipmentID', 'OrderReference', 'OrderDate']
orders[orders.duplicated(subset=order_prim_key, keep=False)]

,OrderReference,CustomerID,ProductID,OrderDate,Sales,Quantity,Discount,Profit,ShipmentID


In [312]:
orders.columns

Index(['OrderReference', 'CustomerID', 'ProductID', 'OrderDate', 'Sales',
       'Quantity', 'Discount', 'Profit', 'ShipmentID'],
      dtype='object')

# Export to database

In [313]:
%store df

Stored 'df' (DataFrame)


In [314]:
from pathlib import Path
db_path = Path(config['processed_data']['database'])
db_path.exists()

True

In [319]:
address_secondary

,index,Country,Region,State
0,0,United States,East,New York
1,1,United States,West,California
2,2,United States,South,Florida
3,3,United States,West,Arizona
4,5,United States,Central,Texas
5,6,United States,South,Virginia
6,7,United States,Central,Michigan
7,10,United States,West,New Mexico
8,11,United States,South,Georgia
9,13,United States,West,Utah


In [326]:
import sqlite3

with sqlite3.connect(config['processed_data']['database']) as conn:
    conn.execute('DELETE FROM orders')
    conn.execute('DELETE FROM customer')
    conn.execute('DELETE FROM product')
    conn.execute('DELETE FROM productcategories')
    conn.execute('DELETE FROM address')
    conn.execute('DELETE FROM addresssecondary')
    conn.execute('DELETE FROM shipment')
    customer.to_sql('customer', conn, if_exists='append', index_label='ID')
    product.to_sql('product', conn, if_exists='append', index_label='ID')
    product_categories.to_sql('productcategories', conn, if_exists='append', index_label='ID')
    address.to_sql('address', conn, if_exists='append', index_label='ID')
    address_secondary.to_sql('addresssecondary', conn, if_exists='append', index_label='ID')
    shipment.to_sql('shipment', conn, if_exists='append', index_label='ID')
    orders.to_sql('orders', conn, if_exists='append', index=False)
    